In [ ]:
# Imports
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text, MetaData
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import random
import time
import warnings
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option("mode.chained_assignment", None)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [3]:
# Log function
log_file_name = 'P:/BatchScheduling/LogFiles/sby_conversion.txt'
run_code = str(int(random.random()*100000))+'# '


def log(s, e=''):
    try:
        if e != '':
            e = ' - Error: ' + str(e)
        log_text = str(datetime.now())[:19] + ' - ' + s + e
        print('\n', log_text)
        f = open(log_file_name, "a")
        f.write('\nRun_'+run_code+log_text)
        f.close()
    except Exception as e:
        print(e)

### Start & End Date

In [4]:
startDate = datetime.today().strftime("'2025-07-01'")
endDate = datetime.today().strftime("'2025-09-30'")
# filDate = datetime(2024,12,1)

log(f'Running script for {startDate} to {endDate}')


 2025-12-15 13:24:06 - Running script for '2025-07-01' to '2025-09-30'


### Standby Activation Data

In [5]:
# SQL queries
trxn_q = f"""SELECT * FROM Trxn_log_FD WITH (NOLOCK) WHERE UTCPAIRINGDATE BETWEEN DATEADD(DAY, -1, {startDate}) AND {endDate} OR (RECTYPE = 6 AND FORDATE BETWEEN DATEADD(DAY, -1, {startDate}) AND {endDate}) ORDER BY ID, UTCPAIRINGDATE, ONDATE, EXACT_TIME"""

codeType_q = "SELECT * FROM Available_days"

pubblanks_q = f"""SELECT B.* FROM
(
SELECT DISTINCT I.StaffNumber, D.DutyDay, 'Blank' PubBlank FROM 
(
SELECT DISTINCT StaffNumber FROM Crew_Roster_details WITH (NOLOCK) WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'P' AND Flt_pos IN (1,2)
)I 
CROSS JOIN 
(
SELECT DISTINCT DutyDay FROM Crew_Roster_details WITH (NOLOCK) WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'P' AND Flt_pos IN (1,2)
)D 
LEFT JOIN 
(
SELECT DISTINCT StaffNumber, DutyDay FROM Crew_Roster_details WITH (NOLOCK) WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'P' AND Flt_pos IN (1,2)
)Cr 
ON I.StaffNumber = Cr.StaffNumber AND D.DutyDay = Cr.DutyDay 
WHERE Cr.StaffNumber IS NULL
)B
LEFT JOIN
(
SELECT DISTINCT StaffNumber, CAST(MAX(ISNULL(DutyDebriefDateLCL, '1980-01-01')) AS date) DutyEnd FROM Crew_Roster_details WITH (NOLOCK)
LEFT JOIN Available_days ON TripCode = Code
WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'P' AND Flt_pos IN (1,2)
GROUP BY StaffNumber, DutyDay
UNION
SELECT DISTINCT StaffNumber, CAST(MAX(ISNULL(DutyDebriefDateLCL, '1980-01-01')) AS date) DutyEnd FROM Crew_Roster_details WITH (NOLOCK)
LEFT JOIN Available_days ON TripCode = Code
WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'A' AND Flt_pos IN (1,2)
GROUP BY StaffNumber, DutyDay
)C
ON C.StaffNumber = B.StaffNumber AND C.DutyEnd = B.DutyDay
WHERE C.StaffNumber IS NULL
"""

actskipdays_q = f"""SELECT DISTINCT StaffNumber, DutyDay, TripCode FROM
(
SELECT * FROM Crew_Roster_details WITH (NOLOCK) WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'A' AND Flt_pos IN (1,2)
)Cr 
LEFT JOIN Available_days A 
ON Code = Tripcode 
WHERE FD_Category IN ('Planned Leaves', 'Admin', 'Long NA', 'Unplanned Leaves', 'Training', 'COVID', 'OFF', 'SMC/SMT/SMD') 
UNION 
SELECT DISTINCT I.StaffNumber, D.DutyDay, 'Blank' FROM
(
SELECT DISTINCT StaffNumber FROM Crew_Roster_details WITH (NOLOCK) WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'A' AND Flt_pos IN (1,2)
)I 
CROSS JOIN 
(
SELECT DISTINCT DutyDay FROM Crew_Roster_details WITH (NOLOCK) WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'A' AND Flt_pos IN (1,2)
)D 
LEFT JOIN 
(
SELECT DISTINCT StaffNumber, DutyDay FROM Crew_Roster_details WITH (NOLOCK) WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'A' AND Flt_pos IN (1,2)
)Cr 
ON I.StaffNumber = Cr.StaffNumber AND D.DutyDay = Cr.DutyDay 
WHERE Cr.StaffNumber IS NULL"""

pairings_q = f"""SELECT TRIM(croute) croute, pairing_startDate, CAST(as_on_date AS date) as_on_date, MAX(pairing_duration) TotalDays, STRING_AGG(TRIM(dep), '-') WITHIN GROUP (ORDER BY std) + '-' + MAX(TRIM(LastArr)) Sectors, SUM(block) TotalBlock, MIN(std) FirstSTD, MAX(sta) LastSTA, STRING_AGG(DHD, '-') WITHIN GROUP (ORDER BY std) SectorType FROM 
(
SELECT ISNULL(DHT_indicator, 'OPR') DHD, LAST_VALUE(arr) OVER (PARTITION BY TRIM(croute), pairing_startDate, as_on_date ORDER BY std RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) LastArr, * FROM pairing_data WITH (NOLOCK)
WHERE pairing_startDate BETWEEN {startDate} AND {endDate} AND CAST(as_on_date AS date) BETWEEN DATEADD(DAY, -2, pairing_startDate) AND DATEADD(DAY, 1, pairing_endDate) AND Publact = 'A'AND CP+FO>0
)A 
GROUP BY croute, pairing_startDate, CAST(as_on_date AS date)"""

headcount_q = f"""SELECT IGA, As_on_date FROM pilotheadcount_datestamp WITH (NOLOCK) WHERE As_on_Date BETWEEN {startDate} AND {endDate} AND (TrainingQualfication IS NULL OR TrainingQualfication IN ('DE', 'INA', 'LTC')) AND STATUS IN ('Line Pilot', 'Trainer', 'Management Pilot')"""

off_q = f"""SELECT DISTINCT P.StaffNumber, P.DutyDay, CONCAT(TRIM(P.FD_Category),' to ',TRIM(A.FD_Category)) CHGTYPE FROM
(
SELECT StaffNumber, DutyDay, FD_Category FROM Crew_Roster_details CR WITH (NOLOCK) 
LEFT JOIN Available_days A ON A.Code = CR.TripCode
WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'P' AND Flt_pos IN (1,2) AND FD_Category = 'OFF'
)P
LEFT JOIN
(
SELECT StaffNumber, DutyDay, ISNULL(FD_Category, 'FDUT') FD_Category FROM Crew_Roster_details CR WITH (NOLOCK) 
LEFT JOIN Available_days A ON A.Code = CR.TripCode
WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'A' AND Flt_pos IN (1,2) AND (FD_Category IS NULL OR FD_Category NOT IN ('Admin','COVID','High Hours','Long NA','OFF','Planned Leaves','Reserved','Resigned','Unplanned Leaves', 'SMC/SMT/SMD'))
)A
ON P.StaffNumber = A.StaffNumber AND P.DutyDay = A.DutyDay
WHERE A.StaffNumber IS NOT NULL"""

offinf_q = f"""SELECT A.StaffNumber AS ID, A.DutyDay AS ISTFORDATE FROM
(
SELECT StaffNumber, DutyDay, FD_Category FROM Crew_Roster_details WITH (NOLOCK)
LEFT JOIN Available_days ON TripCode = Code
LEFT JOIN pilotheadcount_datestamp P WITH (NOLOCK) ON IGA = StaffNumber AND DutyDay = As_on_Date
WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'P' AND Flt_pos IN (1,2) AND FD_Category = 'OFF' AND (TrainingQualfication IS NULL OR TrainingQualfication IN ('DE', 'INA', 'LTC')) AND STATUS IN ('Line Pilot', 'Trainer', 'Management Pilot')
)A
LEFT JOIN
(
SELECT StaffNumber, DutyDay, MAX(ISNULL(DutyDebriefDateLCL, '1980-01-01')) DutyEnd FROM Crew_Roster_details WITH (NOLOCK)
LEFT JOIN Available_days ON TripCode = Code
WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'A' AND Flt_pos IN (1,2) AND ISNULL(ATA_LCL, '1980-01-01') <> '1980-01-01'
GROUP BY StaffNumber, DutyDay
)B
ON A.StaffNumber = B.StaffNumber AND A.DutyDay = DATEADD(DAY, 1, B.DutyDay) AND DATEADD(MINUTE, 60, CAST(A.DutyDay AS datetime)) < B.DutyEnd
WHERE B.StaffNumber IS NOT NULL
ORDER BY A.StaffNumber, A.DutyDay"""

sbysol_q = f"""SELECT DISTINCT A.StaffNumber, B.DutyDay FROM
(
SELECT * FROM Crew_Roster_details WITH (NOLOCK) WHERE DutyDay BETWEEN {startDate} AND {endDate} AND TripCode IN ('SBY', 'SBYP') AND Flt_pos IN (1,2) AND DATEDIFF(MINUTE, DutyReporting_LCL, DutyDebriefDateLCL) < 480 AND CAST(DutyReporting_LCL AS date) <> CAST(DutyDebriefDateLCL AS date)
)A
INNER JOIN
(
SELECT * FROM Crew_Roster_details WITH (NOLOCK) WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'A' AND Flt_pos IN (1,2) AND fleetype IS NOT NULL AND DutyReporting_LCL IS NOT NULL
)B
ON A.StaffNumber = B.StaffNumber AND B.DutyReporting_LCL BETWEEN A.DutyReporting_LCL AND A.DutyDebriefDateLCL
"""

rosterPublishDate_q = f"""SELECT * FROM PublishDates_FD WHERE {startDate} <= RosterEndDate AND {endDate} >= RosterStartDate"""

requested_swapped_q = f"""SELECT R.ID, DATEADD(DAY, LEG_DAY, '1980-01-01') FORDATE FROM ROSTERDT R
INNER JOIN
(
SELECT ID, MIN(POS) POS
FROM CREWQUAL
WHERE POS IN (1,2) AND CAST(GETDATE() AS date) BETWEEN DATEADD(DAY, ACTIVBEG, '1980-01-01') AND DATEADD(DAY, ACTIVEND, '1980-01-01') AND ID < 99999
GROUP BY ID
)CQ ON R.ID = CQ.ID
WHERE DATEADD(DAY, LEG_DAY, '1980-01-01') BETWEEN {startDate} AND {endDate}  AND ((RECTYPE = 21 AND VALUE1 IN (10,15,16,32,33,34,35,36,38,42,43,44)) OR RECTYPE IN (106, 100))
"""

diversion_q = f"""SELECT CR.StaffNumber ID, CR.Dutyday ISTFORDATE FROM
(
SELECT * FROM Crew_Roster_details WITH (NOLOCK) WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'A' AND Flt_pos IN (1,2)
)CR
LEFT JOIN
(
SELECT * FROM DFS WITH (NOLOCK) WHERE FLTDATE BETWEEN {startDate} AND {endDate} AND Status = 'DV'
)F
ON F.FLT = CR.Flt_Num AND F.FLTDATE = CAST(CR.STD_LCL AS date) AND F.DEP = CR.Dep_city AND F.ARR = CR.Arr_city
WHERE F.FLT IS NOT NULL
"""

longna_q = f"""SELECT IGA AS ID FROM Long_NA_pilot WITH (NOLOCK) WHERE MONTH = DATEADD(DAY, 1, EOMONTH({startDate}, -1))"""

dhd_q = f"""SELECT DATEADD(DAY, CDATE, '1980-01-01') UTCPAIRINGDATE, TRIM(CONVERT(CHAR(5), SUBSTRING(CONVERT(BINARY(8), CROUTE), 4, 5))) NEW FROM CREWLEG WITH (NOLOCK) WHERE DATEADD(DAY, DAY, '1980-01-01') BETWEEN DATEADD(DAY, -7, {startDate}) AND {endDate} GROUP BY CDATE, CROUTE HAVING MIN(PAX) = 1"""

publishedFDUT_q = f"""SELECT DISTINCT P.StaffNumber ID, A.DutyDay ISTFORDATE FROM
(
SELECT * FROM Crew_Roster_details WITH (NOLOCK) WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'P' AND Flt_pos IN (1,2) AND ((DutyReporting_LCL IS NOT NULL AND fleetype IS NOT NULL) OR (TripCode IN ('SBY', 'SBYP', 'TSBY')))
)P
INNER JOIN
(
SELECT * FROM Crew_Roster_details WITH (NOLOCK) WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'A' AND Flt_pos IN (1,2) AND ((DutyReporting_LCL IS NOT NULL AND fleetype IS NOT NULL) OR (TripCode IN ('SBY', 'SBYP', 'TSBY')))
)A
ON A.StaffNumber = P.StaffNumber AND A.DutyDay = P.DutyDay AND A.TripCode = P.TripCode
"""

publishedOFF_q = f"""SELECT StaffNumber ID, DutyDay ISTFORDATE FROM Crew_Roster_details WITH (NOLOCK)
LEFT JOIN Available_days A  ON A.Code = TripCode
WHERE DutyDay BETWEEN {startDate} AND {endDate} AND publact = 'P' AND Flt_pos IN (1,2) AND FD_Category IN ('OFF')"""

# sby_codes_q = f"""select * from Available_days with(nolock) where FD_Category = 'SBY'"""

In [6]:
# Load function
def load_data(query, server_name, database_name):
    def get_var_name(var):
        for name, value in globals().items():
            if value is var:
                return name

    query_name = get_var_name(query)

    conn_str = f"mssql+pyodbc://{server_name}/{database_name}?trusted_connection=yes&driver={'ODBC Driver 17 for SQL Server'}"

    try:

        engine = create_engine(conn_str, fast_executemany=True)

        df = pd.read_sql(query, engine)

        log(f"{query_name} query executed successfully and " +

            str(df.shape[0]) + " result stored in DataFrame.")
        return df

    except Exception as e:

        log("Error in query load:", e)

    engine.dispose()

In [7]:
# Data loads from SQL
log('Query load started')

trxn = load_data(trxn_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

codeType = load_data(codeType_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

pubblanks = load_data(pubblanks_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

actskipdays = load_data(actskipdays_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

# pairings = load_data(pairings_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

# headcount = load_data(headcount_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

# off = load_data(off_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

# offinf = load_data(offinf_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

# sbysol = load_data(sbysol_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

rosterPublishDate = load_data(rosterPublishDate_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

requested_swapped = load_data(requested_swapped_q, r'ZPW-AIMS-PDB15.indigo.in,57345', 'IGO')

# diversion = load_data(diversion_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

# longna = load_data(longna_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

# dhd = load_data(dhd_q, r'ZPW-AIMS-PDB15.indigo.in,57345', 'IGO')

# publishedFDUT = load_data(publishedFDUT_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

# publishedOFF = load_data(publishedOFF_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

# sby_codes = load_data(sby_codes_q, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')

log('Query load ended')


 2025-12-15 13:24:06 - Query load started

 2025-12-15 13:25:00 - trxn_q query executed successfully and 2192819 result stored in DataFrame.

 2025-12-15 13:25:00 - codeType_q query executed successfully and 552 result stored in DataFrame.

 2025-12-15 13:25:04 - pubblanks_q query executed successfully and 50425 result stored in DataFrame.

 2025-12-15 13:25:07 - actskipdays_q query executed successfully and 354249 result stored in DataFrame.

 2025-12-15 13:25:07 - rosterPublishDate_q query executed successfully and 3 result stored in DataFrame.

 2025-12-15 13:25:08 - requested_swapped_q query executed successfully and 51196 result stored in DataFrame.

 2025-12-15 13:25:08 - Query load ended


In [8]:
# Initial transformations for all dataframes
# oldPairing = pairings.rename(columns={"croute": "OLD", "pairing_startDate": "OLDUTCPAIRINGDATE", "as_on_date": "OLD_As_on_date1", "TotalDays": "OLDTotalDays", "Sectors": "OLDSectors", "TotalBlock": "OLDTotalBlock", "FirsSTD": "OLDFirstSTD", "LastSTA": "OLDLastSTA", "SectorType": "OLDSectorType", })

# newPairing = pairings.rename(columns={"croute": "NEW", "pairing_startDate": "UTCPAIRINGDATE", "as_on_date": "NEW_As_on_date1", "TotalDays": "NEWTotalDays", "Sectors": "NEWSectors", "TotalBlock": "NEWTotalBlock", "FirsSTD": "NEWFirstSTD", "LastSTA": "NEWLastSTA", "SectorType": "NEWSectorType", })

actskipdays.rename(columns={"StaffNumber": "ID", "DutyDay": "ISTFORDATE"}, inplace=True)
actskipdays = actskipdays.astype({'ISTFORDATE': 'datetime64[ns]'})

pubblanks.rename(columns={"StaffNumber": "ID", "DutyDay": "ISTFORDATE"}, inplace=True)
pubblanks = pubblanks.astype({'ISTFORDATE': 'datetime64[ns]'})

# headcount.rename(columns={"IGA": "ID", "As_on_date": "ISTFORDATE"}, inplace=True)
# headcount = headcount.astype({'ISTFORDATE': 'datetime64[ns]'})

# off.rename(columns={"StaffNumber": "ID", "DutyDay": "ISTFORDATE"}, inplace=True)
# off = off.astype({'ISTFORDATE': 'datetime64[ns]'})

# sbysol.rename(columns={"StaffNumber": "ID", "DutyDay": "ISTFORDATE"}, inplace=True)
# sbysol = sbysol.astype({'ISTFORDATE': 'datetime64[ns]'})

# oldPairing["OLDUTCPAIRINGDATE"] = pd.to_datetime(oldPairing["OLDUTCPAIRINGDATE"])
# newPairing["UTCPAIRINGDATE"] = pd.to_datetime(newPairing["UTCPAIRINGDATE"])

# oldPairing["OLD"] = oldPairing["OLD"].str.strip()
# newPairing["NEW"] = newPairing["NEW"].str.strip()

trxn.loc[trxn["CHGTYPE"] == "Deassignment", "NEW"] = "-"
trxn.loc[trxn["CHGTYPE"] == "Assignment", "OLD"] = "-"

codeType.loc[codeType["Code"] == "SVLB", "FD_Category"] = "SVLB"
codeType.loc[codeType["Code"].isin(['ESD', 'EI21', 'EREF', 'EDTC', 'ELP', 'ELPT', 'ED', 'EDRL', 'ETP', 'ETRG', 'ETPS', 'EDRF', 'NMEA', 'NME', 'EGVD', 'ETRM', 'ETRT', 'EUPR', 'EVOL', 'EELP', 'EUPT', 'EAIT', 'E5F5', 'EFIR', 'ETRF', 'EUPG', 'EGRF', 'ERET', 'EM+F', 'EMHP', 'FOSA']), "FD_Category"] = "E-Training"
codeType = codeType[codeType["FD_Category"] != "Blank"]
codeType.loc[codeType["Code"] == "TSBY", "FD_Category"] = "SBY"
codeType["OLD"] = codeType["Code"]
codeType["NEW"] = codeType["Code"]

oldType = codeType[["OLD", "FD_Category"]].rename(columns={"FD_Category": "OLDTYPE"})
newType = codeType[["NEW", "FD_Category"]].rename(columns={"FD_Category": "NEWTYPE"})

# publishedFDUT['ISTFORDATE'] = pd.to_datetime(publishedFDUT['ISTFORDATE'])
# publishedOFF['ISTFORDATE'] = pd.to_datetime(publishedOFF['ISTFORDATE'])

# dhd['UTCPAIRINGDATE'] = pd.to_datetime(dhd['UTCPAIRINGDATE'])

In [9]:
# diversion['ISTFORDATE'] = pd.to_datetime(diversion['ISTFORDATE'])

In [10]:
# Initialize trxnWorking and create date columns
trxnWorking = trxn[["ID", "UTCPAIRINGDATE", "FORDATE", "ONDATE", "EXACT_TIME", "RECTYPE", "OLD", "NEW", "ODUTCODE", "NDUTCODE", "OLDTIME", "OLDENDTIME", "NEWTIME", "ENDTIME", "FLT", "SWAPID", "CRWREQSWAP", "MODFREASON", "USER_", "NOTUSER", "NOTIFIED"]].copy()

trxnWorking["ISTDUTYTIME"] = np.where((trxnWorking["OLD"] != "-") & (trxnWorking["NEW"] == "-") & (trxnWorking["RECTYPE"] == 4), (trxnWorking["UTCPAIRINGDATE"] + pd.to_timedelta(trxnWorking["OLDTIME"] + 330, unit='min')), (trxnWorking["UTCPAIRINGDATE"] + pd.to_timedelta(trxnWorking["NEWTIME"] + 330, unit='min')))
trxnWorking["ISTDUTYDAY"] = pd.to_datetime(trxnWorking["ISTDUTYTIME"].dt.date)

trxnWorking['ISTFORDATE'] = np.where((trxnWorking['FORDATE'] >= trxnWorking['ISTDUTYDAY']), pd.to_datetime(trxnWorking['FORDATE']), pd.to_datetime(trxnWorking['ISTDUTYDAY']))

In [11]:
# Assign Publish date for each ISTFORDATE and publish flag set
trxnWorking['key'] = 1
rosterPublishDate['key'] = 1

cross_joined = pd.merge(trxnWorking, rosterPublishDate, on='key').drop('key', axis=1)
cross_joined = cross_joined[(cross_joined['ISTFORDATE'] >= cross_joined['RosterStartDate']) & (cross_joined['ISTFORDATE'] <= cross_joined['RosterEndDate'])]

cross_joined = cross_joined.drop_duplicates(subset=['ISTFORDATE'], keep='first')[['ISTFORDATE', 'PublishDate']]

trxnWorking = trxnWorking.merge(cross_joined, on='ISTFORDATE', how='left')

trxnWorking["Publish"] = np.where(trxnWorking["ONDATE"] >= pd.to_datetime(trxnWorking['PublishDate']) + pd.DateOffset(days=1), 1, 0)

trxnWorking["Publish1"] = np.where(trxnWorking["NOTIFIED"] != 2, 1, 0)

In [12]:
# Clean OLD and NEW, and add EXACT_TIME to ONDATE
trxnWorking["OLD"] = trxnWorking["OLD"].str.strip()


trxnWorking["NEW"] = trxnWorking["NEW"].str.strip()

trxnWorking["ONDATE"] = pd.to_datetime(trxnWorking["ONDATE"].dt.date, format="%Y-%m-%d %H:%M:%S.%f") + pd.to_timedelta(trxnWorking["EXACT_TIME"], unit="ms")

trxnWorking = trxnWorking.astype({'UTCPAIRINGDATE': 'datetime64[ns]', 'FORDATE': 'datetime64[ns]', 'ONDATE': 'datetime64[ns]', 'ISTDUTYTIME': 'datetime64[ns]', 'ISTDUTYDAY': 'datetime64[ns]', 'ISTFORDATE': 'datetime64[ns]', 'PublishDate': 'datetime64[ns]'})

In [13]:
# Filter RECTYPE 8
trxnWorking = trxnWorking[trxnWorking['RECTYPE'] != 8]

log('Filtered RECTYPE 8')


 2025-12-15 13:25:22 - Filtered RECTYPE 8


In [14]:
# Merge trxnWorking with pubblanks
trxnWorking = trxnWorking.merge(pubblanks, how="left", on=["ID", "ISTFORDATE"])

In [15]:
# Create PartString and order trxnWorking and reindex
trxnWorking["PartString"] = (trxnWorking["ID"].astype("str") + "-" + trxnWorking["ISTFORDATE"].astype("str"))

sort_columns = ["ID", "ISTFORDATE", "ONDATE"]
trxnWorking = trxnWorking.sort_values(by=sort_columns)
trxnWorking.reset_index(inplace=True, drop=True)

In [16]:
trxnWorkingPreLogic = trxnWorking.copy()

In [17]:
# Initialize stack and required variables and create Transaction function
log('Transactions creation started')

trxnWorking["OLD1"] = ""
trxnWorking["OLDUTCPAIRINGDATE"] = None
trxnWorking["OLDISTPAIRINGDATE"] = None
trxnWorking["OLDTIME1"] = 0
trxnWorking["OLDENDTIME1"] = 0
trxnWorking["OLDONDATE"] = None
trxnWorking["OLDISTFORDATE"] = None
trxnWorking["Assignments"] = None
trxnWorking["Sequence"] = None
trxnWorking["Cancellation"] = 0
trxnWorking["Request"] = 0
trxnWorking["SameCheck"] = 0

stacks = {}


def process_group(df):
    sequence = 1
    stack = []
    last_values = {
        "last_": "-",
        "last_UTCPAIRINGDATE": None,
        "last_TIME": 0,
        "last_ENDTIME": 0,
        "last_ONDATE": None,
        "last_ISTDUTYTIME": None,
        "last_ISTFORDATE": None
    }

    for idx, row in df.iterrows():
        df.at[idx, "Sequence"] = sequence

        if row["RECTYPE"] in [4, 10]:
            if row["OLD"] != "-":
                if row["OLD"] in stack:
                    stack.remove(row["OLD"])
                df.at[idx, "Assignments"] = stack.copy()

            if row["NEW"] != "-":
                stack.append(row["NEW"])
                df.at[idx, "Assignments"] = stack.copy()

            if row["NEW"] != "-" and row["OLD"] == "-":
                for key, value in last_values.items():
                    df.at[idx, key.replace("last_", "OLD")] = value

            if row["NEW"] != "-" and row["OLD"] != "-":
                last_values = {
                    "last_": row["OLD"],
                    "last_UTCPAIRINGDATE": row["UTCPAIRINGDATE"],
                    "last_TIME": row["OLDTIME"],
                    "last_ENDTIME": row["OLDENDTIME"],
                    "last_ONDATE": row["ONDATE"],
                    "last_ISTDUTYTIME": row["ISTDUTYTIME"],
                    "last_ISTFORDATE": row["ISTFORDATE"]
                }
                for key, value in last_values.items():
                    df.at[idx, key.replace("last_", "OLD")] = value

            if row["OLD"] != "-" and row["NEW"] == "-": # and row["OLDTIME"] >= last_values["last_TIME"]:
                last_values = {
                    "last_": row["OLD"],
                    "last_UTCPAIRINGDATE": row["UTCPAIRINGDATE"],
                    "last_TIME": row["OLDTIME"],
                    "last_ENDTIME": row["OLDENDTIME"],
                    "last_ONDATE": row["ONDATE"],
                    "last_ISTDUTYTIME": row["ISTDUTYTIME"],
                    "last_ISTFORDATE": row["ISTFORDATE"]
                }

        sequence += 1

    return df


 2025-12-15 13:25:28 - Transactions creation started


In [18]:
# Process each group by (ID, ISTFORDATE)
try:
    trxnWorking = trxnWorking.groupby(['ID', 'ISTFORDATE']).apply(process_group).reset_index(drop=True)
    trxnWorkingPostLogic = trxnWorking.copy()
    log('Transactions created successfully')
except Exception as e:
    log('Transactions creation failed: ', e)

C:\Users\PIM.YuvrajC\AppData\Local\Temp\5\ipykernel_25312\1128068397.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trxnWorking = trxnWorking.groupby(['ID', 'ISTFORDATE']).apply(process_group).reset_index(drop=True)



 2025-12-15 13:39:57 - Transactions created successfully


In [19]:
# trxnWorking = trxnWorking[trxnWorking['ISTFORDATE'] >= filDate]

In [20]:
# Merge oldType and newType
trxnWorking = trxnWorking.merge(oldType, how="left", left_on="OLD", right_on="OLD")
trxnWorking = trxnWorking.merge(newType, how="left", left_on="NEW", right_on="NEW")

In [21]:
# Fill missing values in OLDTYPE and NEWTYPE
trxnWorking["OLDTYPE"].fillna("-", inplace=True)
trxnWorking["NEWTYPE"].fillna("-", inplace=True)

In [23]:
# Split main DF (trxnWorking) into sub DFs (assignments, deassignments, swaps and changes)
assignments = trxnWorking[(trxnWorking['RECTYPE'].isin([4, 10])) & (trxnWorking['NEW'] != '-') & (trxnWorking['NOTIFIED'] != 2) & (trxnWorking['NEW'] != trxnWorking['OLD'])].copy()

deassignments = trxnWorking[(trxnWorking['RECTYPE'] == 4) & (trxnWorking['NEW'] == '-') & (trxnWorking['OLD'] != '-')].copy()

swaps = trxnWorking[(trxnWorking['RECTYPE'].isin([4, 10])) & (trxnWorking['NEW'] != '-') & (trxnWorking['OLD'] != '-')].copy()

changes = trxnWorking[trxnWorking['RECTYPE'].isin([5, 9])].copy()

In [24]:
assignments.loc[assignments['NEWTYPE'] == '-', 'NEWTYPE'] = 'FDUT'

In [25]:
# Initialize sbycount and filter only one occurence
sbycount = assignments[assignments['NEWTYPE'].isin(['SBY', 'FDUT']) & (assignments['FORDATE'] == assignments['UTCPAIRINGDATE'])].groupby(['ID', 'ISTFORDATE', 'NEWTYPE', 'NEWTIME', 'ENDTIME'], as_index=False).agg(ONDATE_min=('ONDATE', 'min'), ONDATE_max=('ONDATE', 'max'), Count=('ONDATE', 'size'))

sbycount = sbycount[(sbycount['Count'] > 1)]

In [26]:
# Vectorized operations for Request, Cancellation, and SameCheck
trxnWorking["Request"] = ((trxnWorking["RECTYPE"] == 4) & (trxnWorking["RECTYPE"].shift(-1) == 23) & (trxnWorking["ODUTCODE"].shift(-1) == 1) & (trxnWorking["NDUTCODE"].shift(-1).isin([1, 3])) & (trxnWorking["PartString"].shift(-1) == trxnWorking["PartString"])).astype(int)

trxnWorking["Cancellation"] = ((trxnWorking["RECTYPE"] == 4) & (trxnWorking["RECTYPE"].shift(1) == 6) & (trxnWorking["PartString"].shift(1) == trxnWorking["PartString"])).astype(int)

trxnWorking["SameCheck"] = ((trxnWorking["PartString"] == trxnWorking["PartString"].shift(-1)) & (trxnWorking["NEW"] == trxnWorking["NEW"].shift(-1)) & (trxnWorking["ISTDUTYTIME"] == trxnWorking["ISTDUTYTIME"].shift(-1)) & (trxnWorking["OLD"] == trxnWorking["OLD"].shift(-1))).astype(int)

In [27]:
# Filter Actual blank and leaves days
trxnWorking = trxnWorking.merge(actskipdays, how="left", on=["ID", "ISTFORDATE"], indicator=True)

trxnWorking = trxnWorking[trxnWorking["_merge"] == "left_only"].drop(["_merge"], axis=1)

log('Filtered Actual Blank and Leave days')


 2025-12-15 13:40:11 - Filtered Actual Blank and Leave days


In [28]:
# Update OLDTYPE and NEWTYPE for FDUT
trxnWorking.loc[(trxnWorking["OLD"] != "-") & (trxnWorking["OLDTYPE"] == "-") & (trxnWorking["RECTYPE"] == 4), "OLDTYPE"] = "FDUT"
trxnWorking.loc[(trxnWorking["NEW"] != "-") & (trxnWorking["NEWTYPE"] == "-") & (trxnWorking["RECTYPE"] == 4), "NEWTYPE"] = "FDUT"
trxnWorking.loc[trxnWorking["RECTYPE"].isin([5, 9]), ["OLDTYPE", "NEWTYPE"]] = "FDUT"

In [29]:
# Fill missing OLDONDATE values
trxnWorking["OLDONDATE1"] = trxnWorking["OLDONDATE"].fillna(trxnWorking['ONDATE'])
trxnWorking["OLDONDATE"].fillna(trxnWorking["ONDATE"] - pd.DateOffset(days=1), inplace=True)

In [30]:
# Calculate ValidDay
trxnWorking["ValidDay"] = (~((trxnWorking["ONDATE"] < trxnWorking["ISTDUTYTIME"]) & (trxnWorking["ISTDUTYTIME"].dt.date < trxnWorking["ISTFORDATE"].dt.date))).astype(int)

In [31]:
sby_activ = trxnWorking[(trxnWorking['OLDTYPE']=='SBY') & (trxnWorking['NEWTYPE']=='FDUT')].copy()

In [48]:
sby_activ_clipped = sby_activ[['ID','UTCPAIRINGDATE','ISTFORDATE','OLDISTDUTYTIME','ISTDUTYTIME','OLD','NEW']].rename(columns={'ID':'CREW ID','UTCPAIRINGDATE':'Pairing_start_date_UTC','OLDISTDUTYTIME':'SBY_time_IST','ISTDUTYTIME':'FDUT_time_IST','OLD':'Old_duty_code','NEW':'New_duty_code','ISTFORDATE':'Duty_date_IST'}).sort_values(['Pairing_start_date_UTC','Duty_date_IST'], ascending=True).drop_duplicates(['CREW ID','Pairing_start_date_UTC'], keep='first').reset_index(drop=True)

In [33]:
sby_activ_clipped

,CREW ID,Pairing_start_date_UTC,Duty_date_IST,SBY_time_IST,FDUT_time_IST,Old_duty_code,New_duty_code
0,1941,2025-06-30,2025-06-30,2025-06-30 07:00:00,2025-06-30 10:00:00,SBY,49B
1,2998,2025-06-30,2025-06-30,2025-06-30 21:00:00,2025-06-30 23:15:00,SBYP,410BB
2,3364,2025-06-30,2025-06-30,2025-06-30 13:30:00,2025-06-30 14:45:00,SBY,6442
3,3675,2025-06-30,2025-06-30,2025-06-30 21:00:00,2025-06-30 22:05:00,SBYP,4485A
4,4042,2025-06-30,2025-06-30,2025-06-30 09:00:00,2025-06-30 20:20:00,TSBY,CD683
...,...,...,...,...,...,...,...
40688,87633,2025-09-30,2025-10-01,2025-10-01 05:05:00,2025-10-01 04:50:00,SBYP,6427
40689,88483,2025-09-30,2025-10-01,2025-09-30 17:00:00,2025-09-30 17:00:00,SBY,2328
40690,91478,2025-09-30,2025-10-01,2025-09-30 06:10:00,2025-09-30 06:10:00,TSBY,F857
40691,31715,2025-09-30,2025-10-02,2025-10-01 04:55:00,2025-10-01 04:55:00,SBY,397


In [34]:
# sby_activ_clipped.to_csv(r'P:\Projects\Dynamic Standby\Data\Input Data\Standby_activation_data_JulytoSept.csv')

### Roster Data

In [40]:
roster_query = f"""WITH Roster AS(
    SELECT *, FIRST_VALUE(DEP) OVER (PARTITION BY Publact, ID, PairingStartDate, TripCode ORDER BY DutyDay, Seq) PairingStartDEP FROM New_Roster_test WITH (NOLOCK) WHERE PairingStartDate BETWEEN {startDate} AND {endDate} AND Pos IN (1,2) AND Publact = 'A'
)
SELECT * FROM Roster WHERE PairingStartDEP = 'DEL'"""

In [42]:
roster_df = load_data(roster_query, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')


 2025-12-16 11:15:24 - roster_query query executed successfully and 190996 result stored in DataFrame.


### Headcount Data

In [44]:
hc_sby_query = "SELECT IGA, [Rank], CrewBase, Contract, STATUS, TrainingQualfication FROM pilotheadcountdata WITH(NOLOCK)"

In [45]:
hc_df = load_data(hc_sby_query, r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')


 2025-12-16 11:57:26 - hc_sby_query query executed successfully and 7451 result stored in DataFrame.


## Transformations

In [ ]:
# Load data here
roster_data = roster_df.copy()
headcount_data = hc_df.copy()
standby_activation_data = sby_activ_clipped.copy()

In [104]:
for col in ['CWBASE','TripCode','DutyCode','DEP','ARR','FleetType','Subfleet','PairingStartDEP']:
    roster_data[col] = roster_data[col].str.strip()

for col in ['STD']:
    roster_data[col] = pd.to_datetime(roster_data[col])

In [105]:
def window(x):
    if x<4:
        return "0-4"
    if x<8:
        return "4-8"
    if x<12:
        return "8-12"
    if x<16:
        return "12-16"
    if x<20:
        return "16-20"
    return "20-24"

In [106]:
def window_num(x):
    if x<4:
        return 1
    if x<8:
        return 2
    if x<12:
        return 3
    if x<16:
        return 4
    if x<20:
        return 5
    return 6

In [107]:
roster_data['Rank'] = roster_data['Pos'].apply(lambda x: 'CP' if x==1 else ('FO' if x==2 else None))

In [108]:
roster_data['Duty Hour of Day'] = roster_data['STD'].dt.hour

roster_data['Duty Window'] = roster_data['Duty Hour of Day'].apply(window)

roster_data['Duty Window Number'] = roster_data['Duty Hour of Day'].apply(window_num)

In [109]:
# IGO_ac_subfleets = load_data(f"SELECT DISTINCT(Equip) AS '6E Subfleet' FROM LOW WITH(NOLOCK) WHERE Dept_Date BETWEEN {startDate} AND {endDate} AND AC_own = '6E'", r'ZPW-MPP-PDB01.indigo.in,57345', 'MPPFDA')
# IGO_ac_subfleets = list(IGO_ac_subfleets['6E Subfleet'])
IGO_ac_subfleets = ['323', '32D', '32H', '32M', '32P', '32S', '32V', 'AT7', 'ATR']

In [116]:
roster_df_fdut_pairing_count = roster_data[(roster_data['DutyCode']=='FDUT') & (roster_data['Publact']=='A') & (roster_data['Subfleet'].isin(IGO_ac_subfleets)) & (roster_data['FleetType'].isin(['320','321']))].sort_values('STD', ascending=True).drop_duplicates(['ID','PairingStartDate','TripCode'], keep='first')

roster_df_fdut_pairing_count = roster_df_fdut_pairing_count.groupby(['PairingStartDate','PairingStartDEP','Duty Window','Duty Window Number','Rank'], as_index=False).size().rename(columns={'PairingStartDate':'Date','PairingStartDEP':'Station','size':'Pairing Start Count'}).sort_values(['Date','Duty Window Number','Rank']).reset_index(drop=True)

In [ ]:
# Actual Pairing Start Counts in Each Window
roster_df_fdut_pairing_count.head(3)

,Date,Station,Duty Window,Duty Window Number,Rank,Pairing Start Count
0,2025-07-01,DEL,0-4,1,CP,3
1,2025-07-01,DEL,0-4,1,FO,5
2,2025-07-01,DEL,4-8,2,CP,49


In [162]:
for col in ['Old_duty_code','New_duty_code']:
    standby_activation_data[col] = standby_activation_data[col].str.strip()

for col in ['FDUT_time_IST']:
    standby_activation_data[col] = pd.to_datetime(standby_activation_data[col])

In [163]:
standby_activation_data['Duty Hour of Day'] = standby_activation_data['FDUT_time_IST'].dt.hour

standby_activation_data['Duty Window'] = standby_activation_data['Duty Hour of Day'].apply(window)

standby_activation_data['Duty Window Number'] = standby_activation_data['Duty Hour of Day'].apply(window_num)

standby_activation_data['Date'] = standby_activation_data['FDUT_time_IST'].dt.date

In [173]:
headcount_data['Rank2'] = headcount_data['Rank'].apply(lambda x: 'CP' if 'CP' in x else ('FO' if 'FO' in x else None))

In [ ]:
standby_activation_count = standby_activation_data.merge(headcount_data[['IGA','CrewBase','Rank2']].rename(columns = {'IGA':'CREW ID','CrewBase':'Station','Rank2':'Rank'}).drop_duplicates('CREW ID'), how='left', on='CREW ID')

In [179]:
standby_activation_count = standby_activation_count.groupby(['Date','Station','Duty Window','Duty Window Number','Rank'], as_index=False).size().rename(columns={'size':'Standby Activation Count'}).sort_values(['Date','Duty Window Number','Rank']).reset_index(drop=True)

In [ ]:
# Actual Standby Activation Counts in Each Window
standby_activation_count

,Date,Station,Duty Window,Duty Window Number,Rank,Standby Activation Count
0,2025-06-30,AMD,4-8,2,CP,2
1,2025-06-30,BLR,4-8,2,CP,3
2,2025-06-30,BOM,4-8,2,CP,7
3,2025-06-30,CCU,4-8,2,CP,4
4,2025-06-30,COK,4-8,2,CP,1
...,...,...,...,...,...,...
9161,2025-10-01,BOM,4-8,2,FO,2
9162,2025-10-01,CCU,4-8,2,FO,1
9163,2025-10-01,DEL,4-8,2,FO,3
9164,2025-10-01,HYD,4-8,2,FO,5


In [192]:
final_df = roster_df_fdut_pairing_count.merge(standby_activation_count.drop('Duty Window Number', axis=1), how='left', on=['Date','Station','Duty Window','Rank'])
final_df['Standby Activation Count'] = final_df['Standby Activation Count'].fillna(0).astype(int)
final_df = final_df[final_df['Date'].between(pd.to_datetime(startDate).date(), pd.to_datetime(endDate).date())]

In [195]:
final_df.to_csv(r'P:\Projects\SBY Conversions Capture\DEL_SBY_prepared__dummy_data.csv', index=False)